# Tabla de resultados
1. Lectura y todos los subresultados

In [1]:
## Get current work directory to load all the custom modules
from pathlib import Path
import sys
BASE_DIR = Path().absolute().resolve(strict=True).parent
if str(BASE_DIR) not in sys.path:
    sys.path.append(str(BASE_DIR))

In [2]:
DATA_PATH = os.path.join(BASE_DIR, 'data', 'gruas')
print(DATA_PATH)

/workspace/Externos/TimeSeries/BenchMark/data/gruas


In [3]:
import datetime as dt
import pandas as pd
import glob

># 1. Lectura y Formateo de datos

In [4]:
csv_paths1 = glob.glob(os.path.join(DATA_PATH + "/result/**/", '*.csv'))
csv_paths2 = glob.glob(os.path.join(DATA_PATH + "/result/**/**/", '*.csv'))
csv_paths = csv_paths1 + csv_paths2

### MSE - tabla de resultados

In [5]:
# Hyperparameters by model
for i, csv_result in enumerate(csv_paths):
    result = pd.read_csv(csv_result, usecols=['idArticulo', 'hyper', 'model'])
    model_name = result.model.sample().tolist()[0]
    result.drop(['model'], axis=1, inplace=True)
    result.columns = ['idArticulo', model_name]
    if i == 0:
        data_hyper = result
    else:
        data_hyper = pd.merge(data_hyper, result, on='idArticulo', how='outer')
data_hyper = data_hyper.fillna(1e2)
data_hyper = data_hyper.set_index('idArticulo')
data_hyper.head()

,ARIMA,LR,DL,DT,XGB
idArticulo,,,,,
bateriascanrec,"{'ar': 2, 'ii': 0, 'ma': 2}",{'n_lags': 5},"{'n_lags': 12, 'random_state': 1458, 'optimize...","{'n_lags': 3, 'max_depth': 4, 'random_state': ...","{'n_lags': 3, 'max_depth': 2, 'random_state': ..."
filtrodepresio,"{'ar': 3, 'ii': 0, 'ma': 2}",{'n_lags': 2},"{'n_lags': 7, 'random_state': 2564, 'optimizer...","{'n_lags': 8, 'max_depth': 8, 'random_state': ...","{'n_lags': 6, 'max_depth': 3, 'random_state': ..."
filtroderetorn,"{'ar': 5, 'ii': 2, 'ma': 2}",100.0,"{'n_lags': 2, 'random_state': 2845, 'optimizer...","{'n_lags': 7, 'max_depth': 6, 'random_state': ...","{'n_lags': 6, 'max_depth': 5, 'random_state': ..."
soport,"{'ar': 2, 'ii': 1, 'ma': 0}",{'n_lags': 2},"{'n_lags': 7, 'random_state': 1396, 'optimizer...","{'n_lags': 10, 'max_depth': 9, 'random_state':...","{'n_lags': 5, 'max_depth': 6, 'random_state': ..."
tuerc,"{'ar': 4, 'ii': 0, 'ma': 1}",100.0,"{'n_lags': 2, 'random_state': 733, 'optimizer'...","{'n_lags': 9, 'max_depth': 6, 'random_state': ...","{'n_lags': 4, 'max_depth': 4, 'random_state': ..."


In [6]:
# MSE by model
for i, csv_result in enumerate(csv_paths):
    result = pd.read_csv(csv_result, usecols=['idArticulo', 'mse_test', 'model'])
    model_name = result.model.sample().tolist()[0]
    result.drop(['model'], axis=1, inplace=True)
    result.columns = ['idArticulo', model_name]
    if i == 0:
        data = result
    else:
        data = pd.merge(data, result, on='idArticulo', how='outer')
data = data.fillna(1e2)
data.head()

,idArticulo,ARIMA,LR,DL,DT,XGB
0,bateriascanrec,0.269498,0.225336,0.103435,3.003126,1.185389
1,filtrodepresio,0.116904,1.497429,2.075234,0.132017,0.070545
2,filtroderetorn,0.402349,100.000000,0.457432,8.758013,1.912659
3,soport,0.423945,0.343306,0.342493,0.635918,0.294121
4,tuerc,0.409980,100.000000,0.725901,3.082166,1.171904


In [7]:
data = data.set_index('idArticulo')
columns = data.columns.tolist()
min_mse = data.min(axis=1).tolist()

def set_model(x, columns, hyper):
    index = x.tolist().index(x.min())
    hyper = hyper.loc[x.name, columns[index]]
    return pd.Series(index=['model', 'hyper'], data=[columns[index], hyper])
print("="*20 + " RESUMEN MSE " + "="*20)
hyper_model = data.apply(set_model, args=(columns, data_hyper ), axis=1)
data = data.join(hyper_model)
data.to_csv(os.path.join(DATA_PATH, 'resumen_mse.csv'))
data
data.style.background_gradient(cmap='Blues')


==================== RESUMEN MSE ====================


,ARIMA,LR,DL,DT,XGB,model,hyper
idArticulo,,,,,,,
bateriascanrec,0.269498,0.225336,0.103435,3.003126,1.185389,DL,"{'n_lags': 12, 'random_state': 1458, 'optimizer': 'RMSprop', 'learning_rate': 0.0001, 'n_epochs': 150}"
filtrodepresio,0.116904,1.497429,2.075234,0.132017,0.070545,XGB,"{'n_lags': 6, 'max_depth': 3, 'random_state': 2184, 'gamma': 0.9615805068917203, 'n_estimators': 2}"
filtroderetorn,0.402349,100.000000,0.457432,8.758013,1.912659,ARIMA,"{'ar': 5, 'ii': 2, 'ma': 2}"
soport,0.423945,0.343306,0.342493,0.635918,0.294121,XGB,"{'n_lags': 5, 'max_depth': 6, 'random_state': 1653, 'gamma': 0.58033015370564, 'n_estimators': 10}"
tuerc,0.409980,100.000000,0.725901,3.082166,1.171904,ARIMA,"{'ar': 4, 'ii': 0, 'ma': 1}"
arandel,100.000000,0.308968,0.494105,0.470892,0.713500,LR,{'n_lags': 5}
filtrodeair,100.000000,1.810550,100.000000,100.000000,0.108929,XGB,"{'n_lags': 5, 'max_depth': 4, 'random_state': 1202, 'gamma': 0.5849827587465187, 'n_estimators': 5}"
pern,100.000000,0.840912,0.952569,0.586523,1.100408,DT,"{'n_lags': 10, 'max_depth': 8, 'random_state': 1748, 'ccp_alpha': 0.03564701729313271}"
